# 제품 이상여부 판별 프로젝트


In [1]:
import optuna
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)

### 데이터 읽어오기


In [6]:
import pandas as pd

RANDOM_STATE = 110

train_data = pd.read_csv("C:/Users/KimDongyoung/Desktop/git_LGaimers5/Lg_aimers5/Lg_aimers5-1/data/train_data_0816.csv")
test_data = pd.read_csv("C:/Users/KimDongyoung/Desktop/git_LGaimers5/Lg_aimers5/Lg_aimers5-1/data/test_data_0816.csv")

In [7]:
# 공통 변수 리스트
com_variables_train = [
    'target', 'Model.Suffix', 'Workorder', 'WorkMode Collect Result'
    , 'Dispenser_1', 'Dispenser_2', 'Receip_No_Collect_Result'
    , 'Production_Qty_Collect_Result', 'Judge_Value_OK'
    , 'Workorder_0.9', 'Workorder_0.6'
]

com_variables_test = [
    'target', 'Set ID', 'Model.Suffix', 'Workorder'
    , 'WorkMode Collect Result', 'Dispenser_1'
    , 'Dispenser_2', 'Receip_No_Collect_Result'
    , 'Production_Qty_Collect_Result', 'Judge_Value_OK'
    , 'Workorder_0.9', 'Workorder_0.6'
]

In [8]:
# 공정 이름 필터링 후 공통 변수와 결합
def create_dataset(train_data, test_data, process_name, com_variables_train, com_variables_test):
    # 열 이름 필터링
    Process_Desc_col = train_data.filter(like=process_name).columns
    
    # train 데이터셋 생성
    final_columns_train = list(Process_Desc_col) + com_variables_train
    train_dataset = train_data[final_columns_train]
    
    # test 데이터셋 생성
    final_columns_test = list(Process_Desc_col) + com_variables_test
    test_dataset = test_data[final_columns_test]
    
    return train_dataset, test_dataset

# 공통 변수 정의
## com_variables_train = [...]  -> 이전 코드에서 정의한 변수 사용
## com_variables_test = [...]   -> 이전 코드에서 정의한 변수 사용

# 데이터셋 생성
train_data_dam, test_data_dam = create_dataset(train_data, test_data, '_Dam', com_variables_train, com_variables_test)
train_data_fill1, test_data_fill1 = create_dataset(train_data, test_data, '_Fill1', com_variables_train, com_variables_test)
train_data_fill2, test_data_fill2 = create_dataset(train_data, test_data, '_Fill2', com_variables_train, com_variables_test)
train_data_autoclave, test_data_autoclave = create_dataset(train_data, test_data, '_AutoClave', com_variables_train, com_variables_test)

---

## Optuna

lgbm

Best trial: score 0.25225225225225223  

params {'n_estimators': 2686, 
    'num_leaves': 865, 
    'max_depth': 232, 
    'learning_rate': 0.0010322861626506287, 
    'min_child_samples': 54,}  

In [5]:
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import train_test_split

THRESHOLD = 0.3

# 최적의 하이퍼파라미터
best_params = {
    'n_estimators': 2686, 
    'num_leaves': 865, 
    'max_depth': 232, 
    'learning_rate': 0.0010322861626506287, 
    'min_child_samples': 54,
    'random_state': RANDOM_STATE,
    'verbose': -1
}

# 모델 생성
model = LGBMClassifier(**best_params)

# 데이터셋 분할 (재사용)
x_train, x_val, y_train, y_val = train_test_split(
    train_data_dam.drop("target", axis=1),
    train_data_dam["target"],
    test_size=0.2,
    shuffle=True,
    random_state=RANDOM_STATE,
)

# y_train과 y_val을 이진 레이블로 변환
y_train = y_train.apply(lambda x: 1 if x == 'AbNormal' else 0)
y_val = y_val.apply(lambda x: 1 if x == 'AbNormal' else 0)

# 모델 학습
model.fit(x_train, y_train)

# 검증 데이터에 대한 예측
y_val_pred_proba = model.predict_proba(x_val)[:, 1]  # 양성 클래스 확률
y_val_pred = (y_val_pred_proba >= THRESHOLD).astype(int)  # 스레드홀드에 따른 예측

# 평가지표 계산
f1 = f1_score(y_val, y_val_pred, average="binary")
accuracy = accuracy_score(y_val, y_val_pred)
precision = precision_score(y_val, y_val_pred)
recall = recall_score(y_val, y_val_pred)

# 혼동 행렬 계산
conf_matrix = confusion_matrix(y_val, y_val_pred)

# 결과 출력
print(f'F1 Score: {f1}')
print('---')
print('Confusion Matrix:')
print(conf_matrix)  # 혼동 행렬 출력
print('---')
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')

F1 Score: 0.21708185053380782
---
Confusion Matrix:
[[7601   61]
 [ 379   61]]
---
Accuracy: 0.9456924216242903
Precision: 0.5
Recall: 0.13863636363636364


xgb

Best trial: score 0.23407022106631986, 
params {'n_estimators': 2213, 'learning_rate': 0.11735028445102921, 'max_depth': 5,   
'alpha': 0.008053553080773119, 'gamma': 0.0006061064433044652, 'reg_alpha': 0.5380410453451789, 
'reg_lambda': 0.8278020434040504, 'colsample_bytree': 0.8672296781322193, 'subsample': 0.3694461716611997}

In [9]:
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import train_test_split

# 'Normal'과 'AbNormal'을 숫자로 변환
train_data_fill2['target'] = train_data_fill2['target'].map({'Normal': 0, 'AbNormal': 1})

THRESHOLD = 0.3

# 최적의 하이퍼파라미터
best_params = {
    'n_estimators': 910, 
    'learning_rate': 0.1516322549932977, 
    'max_depth': 20, 
    'alpha': 0.003953220971450215, 
    'gamma': 0.0013290323576843753, 
    'reg_alpha': 0.35046775966926846, 
    'reg_lambda': 0.020721087884783395, 
    'colsample_bytree': 0.9070762583460225, 
    'subsample': 0.8592284053859449,
    'random_state': RANDOM_STATE
}

# 모델 생성
model = XGBClassifier(**best_params)

# 데이터셋 분할 (재사용)
x_train, x_val, y_train, y_val = train_test_split(
    train_data_fill1.drop("target", axis=1),
    train_data_fill1["target"],
    test_size=0.2,
    shuffle=True,
    random_state=RANDOM_STATE,
)

# y_train과 y_val을 이진 레이블로 변환
y_train = y_train.apply(lambda x: 1 if x == 'AbNormal' else 0)
y_val = y_val.apply(lambda x: 1 if x == 'AbNormal' else 0)

# 모델 학습
model.fit(x_train, y_train)

# 검증 데이터에 대한 예측
y_val_pred_proba = model.predict_proba(x_val)[:, 1]  # 양성 클래스 확률
y_val_pred = (y_val_pred_proba >= THRESHOLD).astype(int)  # 스레드홀드에 따른 예측

# 평가지표 계산
f1 = f1_score(y_val, y_val_pred, average="binary")
accuracy = accuracy_score(y_val, y_val_pred)
precision = precision_score(y_val, y_val_pred)
recall = recall_score(y_val, y_val_pred)

# 혼동 행렬 계산
conf_matrix = confusion_matrix(y_val, y_val_pred)

# 결과 출력
print(f'F1 Score: {f1}')
print('---')
print('Confusion Matrix:')
print(conf_matrix)  # 혼동 행렬 출력
print('---')
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')

C:\Users\KimDongyoung\AppData\Local\Temp\ipykernel_20476\1719465328.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_fill2['target'] = train_data_fill2['target'].map({'Normal': 0, 'AbNormal': 1})


F1 Score: 0.21580928481806777
---
Confusion Matrix:
[[7391  271]
 [ 354   86]]
---
Accuracy: 0.9228585534435941
Precision: 0.24089635854341737
Recall: 0.19545454545454546


dt

Best trial: score 0.24489795918367346,   
params {'max_depth': 122, 'min_samples_split': 30, 'min_samples_leaf': 21, 'max_features': None,  
'splitter': 'best', 'criterion': 'gini'}

In [66]:
import optuna
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, precision_score, recall_score

# 스레드홀드 설정
THRESHOLD = 0.3

# 하이퍼파라미터 설정 (옵튜나에서 구한 최적의 하이퍼파라미터를 입력하세요)
best_params = {
    'max_depth': 122,  
    'min_samples_split': 30,  
    'min_samples_leaf': 21,  
    'max_features': None,  
    'splitter': 'best', 
    'criterion': 'gini', 
    'random_state': RANDOM_STATE
}

# 데이터셋 분할
x_train, x_val, y_train, y_val = train_test_split(
    train_data_fill1.drop("target", axis=1),
    train_data_fill1["target"].map({'Normal': 0, 'AbNormal': 1}),  # y_val도 숫자로 변환
    test_size=0.2,
    shuffle=True,
    random_state=RANDOM_STATE,
)

# 모델 학습
model = DecisionTreeClassifier(**best_params)
model.fit(x_train, y_train)

# 예측
pred_proba = model.predict_proba(x_val)[:, 1]  # 양성 클래스 확률
pred = (pred_proba >= THRESHOLD).astype(int)  # 스레드홀드에 따른 예측

# 평가 지표 계산
f1 = f1_score(y_val, pred, average="binary")
conf_matrix = confusion_matrix(y_val, pred)
accuracy = accuracy_score(y_val, pred)
precision = precision_score(y_val, pred)
recall = recall_score(y_val, pred)

# 결과 출력
print(f'F1 Score: {f1}')
print('---')
print('Confusion Matrix:')
print(conf_matrix)
print('---')
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')

F1 Score: 0.24489795918367346
---
Confusion Matrix:
[[7500  162]
 [ 356   84]]
---
Accuracy: 0.9360651690940508
Precision: 0.34146341463414637
Recall: 0.19090909090909092


et

Best trial: score 0.2237960339943343,  
params {'n_estimators': 1645, 'max_depth': 32, 'min_samples_split': 8,   
'min_samples_leaf': 1, 'criterion': 'entropy', 'bootstrap': False}

In [39]:
import optuna
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, precision_score, recall_score

# 'Normal'과 'AbNormal'을 숫자로 변환
train_data_fill2['target'] = train_data_fill2['target'].map({'Normal': 0, 'AbNormal': 1})

# 스레드홀드 설정
THRESHOLD = 0.3

# 하이퍼파라미터 설정 (옵튜나에서 구한 최적의 하이퍼파라미터를 입력하세요)
best_params = {
    'n_estimators': 1645,  # 예시 값, 옵튜나 결과로 대체
    'max_depth': 32,  # 예시 값, 옵튜나 결과로 대체
    'min_samples_split': 8,  # 예시 값, 옵튜나 결과로 대체
    'min_samples_leaf': 1,  # 예시 값, 옵튜나 결과로 대체
    'criterion': 'entropy',  # 예시 값, 옵튜나 결과로 대체
    'bootstrap': False,  # 예시 값, 옵튜나 결과로 대체
    'random_state': RANDOM_STATE
}

# 데이터셋 분할
x_train, x_val, y_train, y_val = train_test_split(
    train_data_fill2.drop("target", axis=1),
    train_data_fill2["target"],
    test_size=0.2,
    shuffle=True,
    random_state=RANDOM_STATE,
)

# 모델 학습
model = ExtraTreesClassifier(**best_params)
model.fit(x_train, y_train)

# 예측
pred_proba = model.predict_proba(x_val)[:, 1]  # 양성 클래스 확률
pred = (pred_proba >= THRESHOLD).astype(int)  # 스레드홀드에 따른 예측

# 평가 지표 계산
f1 = f1_score(y_val, pred, average="binary")
conf_matrix = confusion_matrix(y_val, pred)
accuracy = accuracy_score(y_val, pred)
precision = precision_score(y_val, pred)
recall = recall_score(y_val, pred)

# 결과 출력
print(f'F1 Score: {f1}')
print('---')
print('Confusion Matrix:')
print(conf_matrix)
print('---')
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')


C:\Users\juneh\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


F1 Score: 0.2237960339943343
---
Confusion Matrix:
[[7475  187]
 [ 361   79]]
---
Accuracy: 0.9323623796593433
Precision: 0.29699248120300753
Recall: 0.17954545454545454


rf


Best trial: score 0.21507537688442213,  
params {'n_estimators': 1005, 'max_depth': 82, 'min_samples_split': 3,  
'min_samples_leaf': 1, 'criterion': 'gini', 'class_weight': 'balanced'}

In [46]:
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, precision_score, recall_score

# 'Normal'과 'AbNormal'을 숫자로 변환
train_data_fill2['target'] = train_data_fill2['target'].map({'Normal': 0, 'AbNormal': 1})

# 스레드홀드 설정
THRESHOLD = 0.3

# 하이퍼파라미터 설정 (옵튜나에서 구한 최적의 하이퍼파라미터를 입력하세요)
best_params = {
    'n_estimators': 1005,  # 예시 값, 옵튜나 결과로 대체
    'max_depth': 82,  # 예시 값, 옵튜나 결과로 대체
    'min_samples_split': 3,  # 예시 값, 옵튜나 결과로 대체
    'min_samples_leaf': 1,  # 예시 값, 옵튜나 결과로 대체
    'criterion': 'gini',
    'class_weight': 'balanced',
    'random_state': RANDOM_STATE
}

# 데이터셋 분할
x_train, x_val, y_train, y_val = train_test_split(
    train_data_fill1.drop("target", axis=1),
    train_data_fill1["target"].map({'Normal': 0, 'AbNormal': 1}),  # y_val도 숫자로 변환
    test_size=0.2,
    shuffle=True,
    random_state=RANDOM_STATE,
)

# 모델 학습
model = RandomForestClassifier(**best_params)
model.fit(x_train, y_train)

# 예측
pred_proba = model.predict_proba(x_val)[:, 1]  # 양성 클래스 확률
pred = (pred_proba >= THRESHOLD).astype(int)  # 스레드홀드에 따른 예측

# 평가 지표 계산
f1 = f1_score(y_val, pred, average="binary")
conf_matrix = confusion_matrix(y_val, pred)
accuracy = accuracy_score(y_val, pred)
precision = precision_score(y_val, pred)
recall = recall_score(y_val, pred)

# 결과 출력
print(f'F1 Score: {f1}')
print('---')
print('Confusion Matrix:')
print(conf_matrix)
print('---')
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')

C:\Users\juneh\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


F1 Score: 0.21507537688442213
---
Confusion Matrix:
[[7214  448]
 [ 333  107]]
---
Accuracy: 0.9036040483831153
Precision: 0.1927927927927928
Recall: 0.2431818181818182


catboost


Best trial: score 0.23717948717948717,  
params {'iterations': 757, 'learning_rate': 0.050198665725373286, 'depth': 9,  
'l2_leaf_reg': 1.424531204481141, 'random_strength': 9.361327508234833,  
'bagging_temperature': 4.80588064825688, 'border_count': 99,  
'scale_pos_weight': 1.1403833705864026}

In [51]:
import optuna
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, precision_score, recall_score

# 'Normal'과 'AbNormal'을 숫자로 변환
train_data_fill2['target'] = train_data_fill2['target'].map({'Normal': 0, 'AbNormal': 1})

# 스레드홀드 설정
THRESHOLD = 0.3

# 하이퍼파라미터 설정 (옵튜나에서 구한 최적의 하이퍼파라미터를 입력하세요)
best_params = {
    'iterations': 757,  # 예시 값, 옵튜나 결과로 대체
    'learning_rate': 0.050198665725373286,  # 예시 값, 옵튜나 결과로 대체
    'depth': 9,  # 예시 값, 옵튜나 결과로 대체
    'l2_leaf_reg': 1.4245312044811413,  # 예시 값, 옵튜나 결과로 대체
    'random_strength': 9.361327508234833,
    'bagging_temperature': 4.80588064825688,
    'border_count': 99,
    'scale_pos_weight': 1.1403833705864026,
    
    'random_seed': RANDOM_STATE,
    'eval_metric': 'F1',
    'logging_level': 'Silent',
    'boosting_type': 'Plain'
}

# 데이터셋 분할
x_train, x_val, y_train, y_val = train_test_split(
    train_data_fill2.drop("target", axis=1),
    train_data_fill2["target"],
    test_size=0.2,
    shuffle=True,
    random_state=RANDOM_STATE,
)

# 모델 학습
model = CatBoostClassifier(**best_params)
model.fit(x_train, y_train)

# 예측
pred_proba = model.predict_proba(x_val)[:, 1]  # 양성 클래스 확률
pred = (pred_proba >= THRESHOLD).astype(int)  # 스레드홀드에 따른 예측

# 평가 지표 계산
f1 = f1_score(y_val, pred, average="binary")
conf_matrix = confusion_matrix(y_val, pred)
accuracy = accuracy_score(y_val, pred)
precision = precision_score(y_val, pred)
recall = recall_score(y_val, pred)

# 결과 출력
print(f'F1 Score: {f1}')
print('---')
print('Confusion Matrix:')
print(conf_matrix)
print('---')
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')


C:\Users\juneh\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


F1 Score: 0.23717948717948717
---
Confusion Matrix:
[[7552  110]
 [ 366   74]]
---
Accuracy: 0.9412490743026414
Precision: 0.40217391304347827
Recall: 0.16818181818181818


.